# Домашняя работа

Сделайте mapper и reducer, чтобы посчитать среднее и дисперсию оценок за фильм.

Реализация через цикл (предпогаем, что мы не знаем сколько у нас фильмов):

In [2]:
import json


n, mean, M2 = 0, 0.0, 0
for path in Path('imdb-user-reviews').glob('**/*'):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            info = json.load(f)
        score = float(info['movieIMDbRating'])
        n += 1
        delta = score - mean
        mean += delta / n
        M2 += delta * (score - mean)

print(mean, (M2 / n) ** (1/2))

NameError: name 'Path' is not defined

На основе этого кода соберите mapper и reducer:

In [3]:
def mapper(path):
    # Ваш код
    return (score, )


def reducer(score_data1, score_data2):
    #  Ваш код
    return n, mean, M2

In [4]:
from functools import reduce
from joblib import Parallel, delayed
from multiprocessing import Pool
from pathlib import Path

In [5]:
def mapper(path):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            info = json.load(f)
        return (float(info['movieIMDbRating']), )


def reducer(score_data1, score_data2):
    if score_data1 is None and score_data2 is None:
        return None
    elif score_data1 is None:
        return score_data2
    elif score_data2 is None:
        return score_data1
    else:
        scores = []
        if len(score_data1) == 1:
            n, mean, M2 = 0, 0.0, 0
            scores.append(score_data1[0])
        else:
            n, mean, M2 = score_data1
        scores.append(score_data2[0])
        for score in scores:
            n += 1
            delta = score - mean
            mean += delta / n
            M2 += delta * (score - mean)
        return n, mean, M2

In [6]:
%%time
n, mean, M2 = reduce(reducer, map(mapper, Path('imdb-user-reviews').glob('**/*')))
print(mean, (M2 / n) ** (1/2))

8.03 1.0517128885774865
Wall time: 4.83 ms


In [7]:
%%time
n, mean, M2 = reduce(reducer, Parallel(n_jobs=2)(delayed(mapper)(path) for path in Path('imdb-user-reviews').glob('**/*')))
print(mean, (M2 / n) ** (1/2))

8.03 1.0517128885774865
Wall time: 1.23 s


In [8]:
%%time
n, mean, M2 = reduce(reducer, map(mapper, list(Path('imdb-user-reviews').glob('**/*')) * 100_000))
print(mean, (M2 / n) ** (1/2))

8.03000000000032 1.0517128885774973
Wall time: 2min 21s


In [9]:
%%time
n, mean, M2 = reduce(reducer, Parallel(n_jobs=2)(delayed(mapper)(path) for path in list(Path('imdb-user-reviews').glob('**/*')) * 100_000))
print(mean, (M2 / n) ** (1/2))

8.03000000000032 1.0517128885774973
Wall time: 1min 22s


In [10]:
%%time
N = 4
n, mean, M2 = reduce(reducer, Parallel(n_jobs=N)(delayed(mapper)(path) for path in list(Path('imdb-user-reviews').glob('**/*')) * 100_000))
print(mean, (M2 / n) ** (1/2))

8.03000000000032 1.0517128885774973
Wall time: 53.3 s


In [11]:
%%time
N = 6
n, mean, M2 = reduce(reducer, Parallel(n_jobs=N)(delayed(mapper)(path) for path in list(Path('imdb-user-reviews').glob('**/*')) * 100_000))
print(mean, (M2 / n) ** (1/2))

8.03000000000032 1.0517128885774973
Wall time: 50.5 s


In [12]:
%%time
N = 8
n, mean, M2 = reduce(reducer, Parallel(n_jobs=N)(delayed(mapper)(path) for path in list(Path('imdb-user-reviews').glob('**/*')) * 100_000))
print(mean, (M2 / n) ** (1/2))

8.03000000000032 1.0517128885774973
Wall time: 52.4 s


In [13]:
import multiprocessing
multiprocessing.cpu_count()

12

# Тут поменял на 12, т.к. у меня 12 ядер

In [14]:
%%time
N = 12
n, mean, M2 = reduce(reducer, Parallel(n_jobs=N)(delayed(mapper)(path) for path in list(Path('imdb-user-reviews').glob('**/*')) * 100_000))
print(mean, (M2 / n) ** (1/2))

8.03000000000032 1.0517128885774973
Wall time: 49.1 s


In [16]:
%%time
N = 10
n, mean, M2 = reduce(reducer, Parallel(n_jobs=N)(delayed(mapper)(path) for path in list(Path('imdb-user-reviews').glob('**/*')) * 100_000))
print(mean, (M2 / n) ** (1/2))

8.03000000000032 1.0517128885774973
Wall time: 42.5 s


# Итог: эффективнее всего показала себя работа на 10 ядрах